In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics 
from sklearn.model_selection import KFold
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.utils.data as data

## Read Dataset

## Neural Network

In [ ]:

class Net(nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = nn.Linear(n_input, n_hidden)
        self.sigmoid = nn.Sigmoid()
        self.predict = nn.Linear(n_hidden, n_output) 

    def forward(self, x):
        x = self.hidden(x)
        x = self.sigmoid(x)
        x = self.predict(x)
        return x

net = Net(n_input=20, n_hidden=500, n_output=1)
batch_size = 100
epochs = 1000
optimizer = torch.optim.SGD(net.parameters(), lr=1e-4)
loss_func = nn.MSELoss()

mseResults = []
maeResults = []
rmseResults = []
r2Results = []

for i,(train_index, test_index) in enumerate(kf.split(tensor_data)):
	for ep in range(0,epochs):
		xTrain, xTest = tensor_data[train_index], tensor_data[test_index] 
		yTrain, yTest = tensor_y[train_index], tensor_y[test_index]
		yTrainRows = yTrain.numel()
		yTrain = yTrain.reshape((yTrainRows,1))
		
		#Train the model
		prediction = net(xTrain)
		loss = loss_func(prediction, yTrain)
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		
		if ep % 100 == 0:
			print("Iteration: " + str(ep) + " Loss: " + str(loss.item()))
	#Test the model
	prediction = net(xTest)
	yPred = prediction.detach().numpy()
	yTest = yTest.numpy()
	
	
	#Plot the prediction
	plt.scatter(yTest,yPred,c="blue")
	plt.title("KFold Cross Validation Results", fontsize=15)
	plt.xlabel('True Values', fontsize=15)
	plt.ylabel('Predictions', fontsize=15)
	p1 = max(max(yPred), max(yTest))
	p2 = min(min(yPred), min(yTest))
	plt.plot([p1, p2], [p1, p2], 'g-')
	
	#Calculate the mae,mse,rmse, and r2 of that kfold
	maeTest = metrics.mean_absolute_error(yTest, yPred)
	mseTest= metrics.mean_squared_error(yTest, yPred)
	rmseTest = math.sqrt(mseTest)
	r2Test = metrics.r2_score(yTest,yPred)
	
	maeResults.append(maeTest)
	mseResults.append(mseTest)
	rmseResults.append(rmseTest)
	r2Results.append(r2Test)

#Calculate MSE,RMSE,MAE and R^2 of KFolds
maeSum = 0
mseSum = 0
rmseSum = 0
r2Sum = 0
for i in range(0,10):
	maeSum += maeResults[i]
	mseSum += mseResults[i]
	rmseSum += rmseResults[i]
	r2Sum += r2Results[i]

print("The average MAE of the test data is:" + str(maeSum/10))
print("The average MSE of the test data is:" + str(mseSum/10))
print("The average RMSE of the test data is:" + str(rmseSum/10))
print("The average R^2 of the test data is:" + str(r2Sum/10))

#Calculate MSE,RMSE,MAE and R^2 of Theoretical
maeTheory = metrics.mean_absolute_error(yExp, yTheory)
mseTheory = metrics.mean_squared_error(yExp, yTheory)
rmseTheory = math.sqrt(mseTheory)
r2Theory = metrics.r2_score(yExp,yTheory)

print("The MAE of the theoretical data is:" + str(maeTheory))
print("The MSE of the theoretical data is:" + str(mseTheory))
print("The RMSE of the theoretical data is:" + str(rmseTheory))
print("The R^2 of the theoretical data is:" + str(r2Theory))

#Plot the actual data as line and show
plt.show()